# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [96]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV


nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\camer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\camer\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\camer\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [42]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table("DisasterResponse", engine)
X = df["message"]
Y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [43]:
def tokenize(text: str):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', "urlplaceholder", text)

    words = word_tokenize(text)

    lemmatizer = WordNetLemmatizer()
    cleaned_tokens = []
    [cleaned_tokens.append(lemmatizer.lemmatize(word).strip()) for word in words]
    
    return cleaned_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [27]:
pipeline = Pipeline([
    ("count_vect", CountVectorizer(tokenizer = tokenize)),
    ("tfidf", TfidfTransformer()),
    ("clf", MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state = 42)

pipeline.fit(X_train, y_train)

Pipeline(steps=[('count_vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000022DE4694670>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [47]:
y_pred = pipeline.predict(X_test)

In [98]:
def display_results(y_pred, y_test, dense: bool = False):
    avg_model_score = np.asarray([0, 0, 0], dtype = np.float32)

    for i in range(y_pred.shape[1]):
        report = classification_report(y_test.iloc[:, i], pd.DataFrame(y_pred).iloc[:, i], output_dict=True, zero_division=0)['weighted avg']
    
        avg_model_score[0] += report['f1-score']
        avg_model_score[1] += report['precision']
        avg_model_score[2] += report['recall']

        if not dense:
            print(y_test.columns[i].capitalize() + ":")
            print(f"f1 score: {round(report['f1-score'], 2)}, precision: {round(report['precision'], 2)}, recall: {round(report['recall'], 2)}")
            print()

    avg_model_score = avg_model_score / y_pred.shape[1]
    print("Overall Model Performance:")
    print(f"f1 score: {avg_model_score[0]}, precision: {avg_model_score[1]}, recall: {avg_model_score[2]}")

display_results(y_pred, y_test)

Related:
f1 score: 0.76, precision: 0.78, recall: 0.79

Request:
f1 score: 0.87, precision: 0.89, recall: 0.89

Offer:
f1 score: 0.99, precision: 0.99, recall: 0.99

Aid_related:
f1 score: 0.77, precision: 0.77, recall: 0.77

Medical_help:
f1 score: 0.89, precision: 0.91, recall: 0.92

Medical_products:
f1 score: 0.93, precision: 0.94, recall: 0.95

Search_and_rescue:
f1 score: 0.97, precision: 0.97, recall: 0.98

Security:
f1 score: 0.97, precision: 0.97, recall: 0.98

Military:
f1 score: 0.96, precision: 0.96, recall: 0.97

Child_alone:
f1 score: 1.0, precision: 1.0, recall: 1.0

Water:
f1 score: 0.94, precision: 0.95, recall: 0.95

Food:
f1 score: 0.91, precision: 0.92, recall: 0.92

Shelter:
f1 score: 0.92, precision: 0.93, recall: 0.93

Clothing:
f1 score: 0.98, precision: 0.98, recall: 0.99

Money:
f1 score: 0.97, precision: 0.98, recall: 0.98

Missing_people:
f1 score: 0.98, precision: 0.99, recall: 0.99

Refugees:
f1 score: 0.95, precision: 0.96, recall: 0.97

Death:
f1 score: 

### 6. Improve your model
Use grid search to find better parameters. 

In [97]:
parameters = {
        'count_vect__ngram_range': ((1, 1), (1, 2)),
        'clf__estimator__n_estimators': [50, 100, 200],
        'clf__estimator__min_samples_split': [2, 3, 4]
    }

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [99]:
cv.fit(X_train, y_train)

KeyboardInterrupt: 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.